In [ ]:
import compyute as cp

# Example 3.2

### Deep neural network using a custom model

The goal of this model is to classify iris species based on numerical features, but this time the model and the training loop are written "from scratch".

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/uciml/iris and place it into the *data* directory.

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd
from compyute.preprocessing import normalize, split_train_val_test


data_orig = pd.read_csv('../data/iris.csv')
data = data_orig.copy()
data.drop(columns=['Id'], inplace=True)

data["Species"] = data["Species"].astype("category").cat.codes

data_tensor = cp.tensor(data.to_numpy())
train, val, test = split_train_val_test(data_tensor, ratio_val=0.25, ratio_test=0.25)

X_train, y_train = train[:, :-1], train[:, -1].int()
X_val, y_val = val[:, :-1], val[:, -1].int()
X_test, y_test = test[:, :-1], test[:, -1].int()

X_train = normalize(X_train, axis=0)
X_val = normalize(X_val, axis=0)
X_test = normalize(X_test, axis=0)

print (f'{X_train.shape=}')
print (f'{y_train.shape=}')

print (f'{X_val.shape=}')
print (f'{y_val.shape=}')

print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

### Step 2: Build a custom model
If you want to define very specifically what the forward and backward pass of your model should look like, you can build your own custom model instead of using the predefined sequential model.

In [ ]:
import compyute.nn as nn

class MyCustomModel(nn.Container):
    def __init__(self):
        super().__init__()

        # define your layers
        self.lin1 = nn.Linear(4, 16)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(16, 3)

    def forward(self, x):
        # define the forward pass
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)

        # define the backward pass
        if self.training:
            def backward(dy):
                dy = self.lin2.backward(dy)
                dy = self.relu.backward(dy)
                dy = self.lin1.backward(dy)
                return dy
            self._backward = backward
        
        return x

model = MyCustomModel()

In [ ]:
model.summary(input_shape=(4,))

### Step 3: Train the model

You can also write a custom training loop instead of using the `Trainer`.

In [ ]:
epochs = 10000
batch_size = X_train.shape[0]

train_dl = nn.DataLoader(X_train, y_train, batch_size)
val_dl = nn.DataLoader(X_val, y_val, batch_size)
loss_func = nn.CrossEntropy()
optim = nn.optimizers.SGD(model.parameters)

for e in range(epochs):
    
    # training
    model.set_training(True)
    for x, y in train_dl():
        # forward pass
        y_pred = model(x)
        _ = loss_func(y_pred, y)

        # backward pass
        optim.reset_grads()  # reset all gradients
        model.backward(loss_func.backward())  # compute new gradients
        optim.step()  # update parameters
    
    # validiation
    model.set_training(False)
    val_loss = 0
    for x, y in val_dl():
        y_pred = model(x)
        val_loss += loss_func(y_pred, y).item()
    val_loss /= len(val_dl)
    print(f"epoch {e}: {val_loss=:.4f}")